In [9]:
[[1,2,3],[2,3,4],[5,3,2]][0:2] == [[1,2,3],[2,3,4]] 

True

In [1]:
from tqdm import tqdm_notebook
import random
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action='ignore') 



data = pd.read_csv('x.csv')
y = pd.read_csv('y.csv')
max_num = 1100
min_num = 900
data['T'] = y
tabu_list = []

tag_tabu = []
base_score = data['T'].mean()
all_t = []

In [2]:
def good_neighbor_sol(sol,num):
    tot_sol = []
    t = sol.copy()
    a = len(sol)
    tag = []
    for i in sol:
        tag.append(i[1])
    for i in range(num):    
        for j in range(a):
            
            d = True
            while(d):
                tm = Make_insol(1)[0]
                if tm[1] not in tag:
                    sol[j] = tm
#                     print('sol:',sol)
                    tot_sol.append(sol)
                    sol = t.copy()
#                     print('sol2:',sol)
                    d = False
    return tot_sol

In [3]:
def check_nbr(nbr, data,desire) : 
    new_nbr = []
    for i in nbr : 
        a, b = Select_data(i,data)
        if a > desire : 
            new_nbr.append(i)
    return new_nbr

In [4]:
def find_local_sol(nbr):
    tmsc = []
    tmso = []
    for i in nbr:
        a,b = Select_data(i,data)
        if min_num<=b<=max_num:
            tmsc.append(a)
            tmso.append(i)
        else : all_t.append(i)
    if len(tmsc):
        score = max(tmsc)
        sol = tmso[tmsc.index(score)]
        return sol
    else:return 0

In [5]:
def Make_valneighbor(sol,num):
    tot_num_list = []
    nbr_sol = []
    tem = []
    for i in range(len(sol)):
        tem.append(sol[i][2])
    tot_num_list.append(tem)
    nbr = []
    num1 = num
    while(num) :
        num-=1
        flg = True
        cnt = 0
        while(flg):
            cnt+=1
            tem_ne = []
            for i in range(len(sol)):
                tem_ne.append(np.random.randint(1,11))
            if (tem_ne not in tot_num_list) or cnt ==5:
                flg = False
                nbr.append(tem_ne)
                tot_num_list.append(tem_ne)
    for i in range(num1):
        nbr_sol.append([])
     
        for j in range(len(sol)):
          
            nbr_sol[i].append([sol[j][0],sol[j][1],nbr[i][j]])
#     print(nbr_sol)
    return(nbr_sol)

In [6]:
def Make_neighbor_sol(sol,num):
    tot_sol = []
#     tot_sol.append(sol)
    a = len(sol)
    for i in range(num):
        t = Make_insol(a)
#         if t[:3] not in tag_num_tabu:
        tot_sol.append(t)
        
    return tot_sol

In [7]:
def Make_neighbor_sol2(sol,num):
    tot_sol = []
#     tot_sol.append(sol)
    a = len(sol)
    for i in range(num):
        t = Make_insol(a)
        if t[:2] not in tag_num_tabu:
            tot_sol.append(Make_insol(a))
        
    return tot_sol

In [8]:
def Select_data(sol,data):
    local_data = data.copy()
    for i in range(len(sol)):
        if sol[i][0] :
            new_data = local_data[data.iloc[:,sol[i][1]] >= sol[i][2]]
        else:
            new_data = local_data[data.iloc[:,sol[i][1]] < sol[i][2]]
        local_data = new_data
    score = new_data['T'].mean()
    len_data = len(new_data)
    return score, len_data

In [9]:
def Make_detail_neighbor(sol,n):
    tot = []
    b = good_neighbor_sol(sol,7*n)
    for i in b:
        tot += Make_valneighbor(i,7*n)
    return tot

In [10]:
def Make_insol(sol_num):
    sol = []
    for i in range(sol_num):
        if i ==0:
            d = [1,np.random.randint(50),np.random.randint(1,11)]
            sol.append(d)
#             tag_num_tabu.append(d[:2])
        else:
            d = [np.random.randint(2),np.random.randint(50),np.random.randint(1,11)]
            sol.append(d)
#             tag_num_tabu.append(d[:2])
    t = []
    for i in range(len(sol)):
        t.append(sol[i][:2])
    if t not in tag_num_tabu:
        tag_num_tabu.append(t)
        return sol
    else : 
        Make_insol(sol_num)

In [11]:
tag_num_tabu = []

In [12]:
start = time.time()


graph_dot = []
tag_num_tabu = []

glo_sol = []
glo_scr = []
desire = base_score
for i in tqdm_notebook(range(1,6)):
    
    
    tag_num_tabu = []
    sol_l = []
    sol_s = []
    real_sol = []
    real_scr = []
    flg=True
    cnt = 0
    cnt2 = 0
    cnt3 = 0
    while(flg):
        cnt+=1
        print('i는',i)
        if cnt == 1:
            insol = Make_insol(i)
            nbr = Make_neighbor_sol(insol,40*(i*i))
            for q in nbr:
                t = Make_detail_neighbor(q,i)
                nbr+=t
                
            b = Make_detail_neighbor(insol,i)
            nbr+=b
            
        else:
            if len(sol_s):
                insol = sol_l[sol_s.index(max(sol_s))]
                nbr = Make_neighbor_sol(insol,40*(i*i))
                for q in nbr:
                    t = Make_detail_neighbor(q,i)
                    nbr+=t
                b = Make_detail_neighbor(insol,i)
                nbr+=b
#                 b = good_neighbor_sol(insol,10*i)
#                 nbr+= b
            else : 
                insol = Make_insol(i)
                nbr = Make_neighbor_sol(insol,40*(i*i))
                for q in nbr:
                    t = Make_detail_neighbor(q,i)
                    nbr+=t
                b = Make_detail_neighbor(insol,i)
                nbr+=b
#                 b = good_neighbor_sol(insol,10*i)
#                 nbr+= b
        sol_l = []
        sol_s = []
        #######여기서 nbr 열망함수 적용 
        nbr = check_nbr(nbr,data,desire)
        for j in tqdm_notebook(nbr):
            
            b_nbr = Make_valneighbor(j,5*(i))
            b_nbr = check_nbr(b_nbr,data,desire)
            sol = find_local_sol(b_nbr)
            #######여기 벨류해???
            if sol : 
                
                a,b = Select_data(sol,data)
                graph_dot.append(a)
                sol_l.append(sol)
                sol_s.append(a)
                
        if len(sol_s):
            s = max(sol_s)
            if cnt >= 2 :
                if max(glo_scr) >= s:
                    cnt2 += 1
                    if cnt2 == 3:
                        flg =False
                else:
                    cnt2 = 0
                print(cnt2)
            glo_sol.append(sol_l[sol_s.index(s)])
            glo_scr.append(s)
            desire = (sum(glo_scr)+base_score) / (len(glo_scr)+1)
        else:
            cnt3+=1
            if cnt3 == 5:
                flg = False
            continue
            
        print(max(glo_scr))
        print(glo_sol[glo_scr.index(max(glo_scr))])
print("time :", time.time() - start)

i는 1



TypeError: 'NoneType' object is not subscriptable

In [ ]:
q_plot = []
x_plot = []
max_plt = []
mean_plt=[]
for i in range(len(graph_dot)):
    q_plot.append(graph_dot[i])
    x_plot.append(i)
    max_plt.append(max(q_plot))
    mean_plt.append(0.05793219386042783)
plt.figure()
plt.plot(x_plot, q_plot,label = 'sol')
plt.plot(x_plot, max_plt, label = 'Max_sol')
plt.plot(x_plot, mean_plt,label = 'All_mean',linestyle = '--')

plt.xlabel('neighbor_solution')
plt.ylabel('Score')
plt.title('final_model')
plt.legend()
plt.show()

In [ ]:
for i in range(len(glo_sol)):
    
    str_a = ''
    str_tm = ''
    a,b = Select_data(glo_sol[i],data)
    for j in range(len(glo_sol[i])):
        q = glo_sol[i][j]
        
        t = ''
        if q[0]==1:
            t = '*'
        else:
            t = '-'
        str_tm+= t
        str_tm+=' ['
        str_tm+='D'
        str_tm+= str(q[1])
        str_tm+= ' , '
        str_tm+= str(q[2])
        str_tm+='] '
    str_a += str_tm
    print('     스코어 : ',round(a,6),      '     NTU:', b,'해 : '  ,str_a, )